In [3]:
import pandas as pd
from tqdm.notebook import tqdm
import re
import json
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
stopwords = stopwords.words('english')
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/arjunth2001/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
def clean_text(x):
    x = str(x).lower()
    x = remove_emoji(x)
    x = re.sub("@[A-Za-z0-9]+","", x) # remove user IDs
    x = re.sub("(http://.*?\s)|(http://.*)",'',str(x)) #remove http links in the text
    x = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)) # remove IP addresses
    x = re.sub('[!"#$%&\()*+,.-/:;<=>?@[\\]^_`{|}~]','',str(x)) #remove all punctuation
    x = x.split(" ")
    x = [i for i in x if i not in stopwords]
    x = " ".join(x)
    x = ' '.join(re.sub("[\'\’\.\,\!\?\:\;\-\=]", " ", x).split())
    # x is a cleaned string please lemmatise it..
    # return value is a string
    return x

In [6]:
train = pd.read_csv("unclean/train.csv")
test = pd.read_csv("unclean/test.csv")
val = pd.read_csv("unclean/val.csv")

In [7]:
train["heading"] = train["heading"].progress_apply(lambda x: clean_text(x))
train["body"] = train["body"].progress_apply(lambda x: clean_text(x))
test["heading"] = test["heading"].progress_apply(lambda x: clean_text(x))
test["body"] = test["body"].progress_apply(lambda x: clean_text(x))
val["heading"] = val["heading"].progress_apply(lambda x: clean_text(x))
val["body"] = val["body"].progress_apply(lambda x: clean_text(x))

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/11707 [00:00<?, ?it/s]

  0%|          | 0/11707 [00:00<?, ?it/s]

  0%|          | 0/11706 [00:00<?, ?it/s]

  0%|          | 0/11706 [00:00<?, ?it/s]

In [9]:
train.to_csv("clean/train.csv",index=False)
test.to_csv("clean/test.csv",index=False)
val.to_csv("clean/val.csv",index=False)